In [19]:
import time
import pycountry
import requests
import json
import pandas as pd
from config import base_currency
from config import comparison_currencies
from api_keys import curr_exchange_api


currency = []
date = []
open_rate = []
close_rate = []


In [5]:

for from_currency in comparison_currencies:

    url = f"https://www.alphavantage.co/query?function=FX_MONTHLY&from_symbol={from_currency}&to_symbol={base_currency}&apikey=curr_exchange_api"
    try:
        response = requests.get(url).json()
    
        for record_date in response["Time Series FX (Monthly)"]:
            currency.append(from_currency)
            date.append(record_date)
            open_rate.append(response["Time Series FX (Monthly)"][record_date]["1. open"])
            close_rate.append(response["Time Series FX (Monthly)"][record_date]["4. close"])

    except:
        print(f"missing data: {from_currency} API: {curr_exchange_api}")
        print(json.dumps(response, indent=4))
        
    time.sleep(60)
        
            


In [6]:
exchange_data = pd.DataFrame({"currency": currency, "end_date": date, "open rate": open_rate, "close rate": close_rate})
exchange_data.head(30)

exchange_data.to_csv("output_file.csv")

In [11]:
exchange_by_country = exchange_data.groupby("currency").count()
exchange_by_country.to_csv("xc_bycountry.csv")

In [13]:
exchange_by_date = exchange_data.groupby("end_date").count()
exchange_by_date.to_csv("xc_bydate.csv")

In [16]:
analysis_data = exchange_data.loc[exchange_data["end_date"] >= "2014-12-31"]

In [18]:
analysis_data.groupby("end_date").count()

,currency,open rate,close rate
end_date,,,
2014-12-31,10,10,10
2015-01-30,10,10,10
2015-02-27,10,10,10
2015-03-31,10,10,10
2015-04-30,10,10,10
...,...,...,...
2020-09-30,10,10,10
2020-10-30,10,10,10
2020-11-30,10,10,10
